In [1]:
import os
from PIL import Image
import tensorflow as tf
import tensorflow.contrib.slim.nets as nets

In [49]:
os.getcwd()

'F:\\CZR\\ANU courses\\2019 S1\\ENGN4528\\project\\tensorflow_cnn'

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)

In [63]:
# 图片路径，两组标签都在该目录下
cwd = r".\resize224"
 
# tfrecord文件保存路径
file_path = r".\resize224"
 
# 每个tfrecord存放图片个数
bestnum = 100
 
# 第几个图片
num = 0
 
# 第几个TFRecord文件
recordfilenum = 0
 
# 将labels放入到classes中
classes = []
for i in os.listdir(cwd):
    classes.append(i)
 
# tfrecords格式文件名
ftrecordfilename = ("traindata_cat&dog.tfrecords-%.3d" % recordfilenum)
writer = tf.python_io.TFRecordWriter(os.path.join(file_path, ftrecordfilename))

In [64]:
for index, name in enumerate(classes):
    class_path = os.path.join(cwd, name)
    for img_name in os.listdir(class_path):
        num = num + 1
        if num > bestnum:    #超过100，写入下一个tfrecord
            num = 1
            recordfilenum += 1
            ftrecordfilename = ("traindata_cat&dog.tfrecords-%.3d" % recordfilenum)
            writer = tf.python_io.TFRecordWriter(os.path.join(file_path, ftrecordfilename))
        
        img_path = os.path.join(class_path, img_name)  # 每一个图片的地址
        img = Image.open(img_path, 'r')
        img_raw = img.tobytes()  # 将图片转化为二进制格式
        example = tf.train.Example(
            features=tf.train.Features(feature={
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
                'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
            }))
        writer.write(example.SerializeToString())  # 序列化为字符串
writer.close()

In [4]:
def read_and_decode_tfrecord(filename):
    filename_deque = tf.train.string_input_producer(filename)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_deque)
    features = tf.parse_single_example(serialized_example, features={
        'label': tf.FixedLenFeature([], tf.int64),
        'img_raw': tf.FixedLenFeature([], tf.string)})
    label = tf.cast(features['label'], tf.int32)
    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.reshape(img, [224, 224, 3])
    img = tf.cast(img, tf.float32) / 255.0      #将矩阵归一化0-1之间
    return img, label

In [66]:
os.listdir(r".\resize224")

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 'traindata_cat&dog.tfrecords-000',
 'traindata_cat&dog.tfrecords-001',
 'traindata_cat&dog.tfrecords-002',
 'traindata_cat&dog.tfrecords-003',
 'traindata_cat&dog.tfrecords-004',
 'traindata_cat&dog.tfrecords-005',
 'traindata_cat&dog.tfrecords-006',
 'traindata_cat&dog.tfrecords-007',
 'traindata_cat&dog.tfrecords-008',
 'traindata_cat&dog.tfrecords-009',
 'traindata_cat&dog.tfrecords-010',
 'traindata_cat&dog.tfrecords-011']

In [5]:
train_list = ['traindata_cat&dog.tfrecords-000',
 'traindata_cat&dog.tfrecords-001',
 'traindata_cat&dog.tfrecords-002',
 'traindata_cat&dog.tfrecords-003',
 'traindata_cat&dog.tfrecords-004',
 'traindata_cat&dog.tfrecords-005',
 'traindata_cat&dog.tfrecords-006',
 'traindata_cat&dog.tfrecords-007',
 'traindata_cat&dog.tfrecords-008',
 'traindata_cat&dog.tfrecords-009',
 'traindata_cat&dog.tfrecords-010',
 'traindata_cat&dog.tfrecords-011']

In [6]:
save_dir = r"./train_cat&dog.model"  # 模型保存路径
batch_size_ = 2
lr = tf.Variable(0.0001, dtype=tf.float32)  # 学习速率
x = tf.placeholder(tf.float32, [None, 224, 224, 3])  # 图片大小为224*224*3
y_ = tf.placeholder(tf.float32, [None])

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
img, label = read_and_decode_tfrecord(train_list)
img_batch, label_batch = tf.train.batch([img, label], num_threads=2, batch_size=batch_size_, capacity=10000)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecor

In [8]:
# 将label值进行onehot编码
one_hot_labels = tf.one_hot(indices=tf.cast(y_, tf.int32), depth=6)

In [9]:
from __future__ import division, print_function, absolute_import

In [10]:
# Training Parameters
learning_rate = 0.001
num_steps = 1
batch_size = 40
display_step = 10

# Network Parameters
num_input = 50176 # MNIST data input (img shape: 28*28)
num_classes = 6 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [11]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=4):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 224, 224, 3])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=4)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=4)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [12]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 3, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([14*14*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=y_))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
# Start training
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:

    # Run the initializer
    sess.run(init)
    with tf.device('/gpu:0'):
        batch_x, batch_y = sess.run([img_batch, label_batch])
        batch_y=sess.run(one_hot_labels, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
        # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={x: batch_x,
                                                                    y_: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc))

        print("Optimization Finished!")